# Data Cleaning / Wrangling

**Goal** : gather all the data in one clean csv file


## Step 1 : Consumptions


In [19]:
import pandas as pd

In [ ]:
consumptions = pd.read_csv("./data/eco2mix-regional-cons-def.csv", delimiter=";")

In [3]:
# keep only interesting columns

In [6]:
# Check column types

In [ ]:
# Convert column types if needed

In [ ]:
# Check for duplicates

In [ ]:
# Remove duplicates

In [ ]:
# Rename columns

In [ ]:
# Check days with missing half hours

We have days with less that 48 half hours, **interesting** !

## Step 2 : Temperatures

In [4]:
weather = pd.read_csv("./data/meteo-paris.csv")

## Step 3 : Merge everything together

[Documentation on how to merge with pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)


![How to merge](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/join-types-merge-names.jpg)


**Inner Merge / Inner join** – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right dataframes.

**Left Merge / Left outer join** – Keep every row in the left dataframe. Where there are missing values of the “on” variable in the right dataframe, add empty / NaN values in the result.

**Right Merge / Right outer join** – Keep every row in the right dataframe. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.

**Outer Merge / Full outer join** – A full outer join returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with NaNs elsewhere.

# Step 4 : handle missing half hours

# Step 4 : interpolate missing values
We want to keep our historical consumptions (which are precious) so we will interpolate missing values for temperature

First question : where are missing values ?

# That's Clean !